# 🧹 **Overview**

This notebook presents a full inspection and preprocessing pipeline for the raw dataset provided by the DAC PRS ITS 2024 Preliminary Round competition. The dataset is originally distributed in a fragmented structure, where the training labels, weather information, metadata, and multi-year solar irradiance files are kept separate. Before constructing any predictive model, these sources need to be explored, validated, and merged into a consistent framework.

The competition's target variable is **% Baseline**, defined as *the percentage of energy generated within one hour relative to the total energy storage capacity*. Since the train and test files contain only timestamps and baseline values, we must incorporate additional contextual features from the supporting datasets in order to build meaningful predictive models. This notebook guides the full process of inspecting the raw inputs and preparing them for downstream modeling.

# 📚 **Library and Configuration**

In this section we load essential Python libraries used throughout the notebook. `pandas` is used for dataframe manipulation, `numpy` for numerical operations, and auxiliary utilities such as `chardet` are required to detect non-standard file encodings encountered in the metadata. Several helper modules for datetime handling, and structured logging are also included to support the data inspection workflow.

In [261]:
# System & Environment Configuration
import sys
sys.path.append("..")

# Ignore warning
from warnings import filterwarnings
filterwarnings("ignore")

# Core Library
import chardet
import numpy as np
import pandas as pd
from pathlib import Path

pd.set_option('display.max_colwidth', None)

# Missing value inspection helper
def column_with_missing(df):
    return df.columns[df.isnull().any()]

# .csv Paths
RAW_ROOT = Path('../data/raw/')
PROCESSED_ROOT = Path('../data/processed/')
METADATA_PATH = RAW_ROOT/'metadata.csv'

# Train and Test
TRAIN_PATH = RAW_ROOT/'train.csv'
TEST_PATH = RAW_ROOT/'test.csv'

# Train and Test Processed
TRAIN_PATH_PROCESSED = PROCESSED_ROOT/'train.csv'
TEST_PATH_PROCESSED = PROCESSED_ROOT/'test.csv'

# Features
WEATHER_PATH = RAW_ROOT/'Weather.csv'
SOLAR_IRRADIANCE = RAW_ROOT/'solar-irradiance'

# Submission
SAMPLE_SUBMISSION = RAW_ROOT/'sample_submission.csv'

print('library and configuration ready!')

library and configuration ready!


In [245]:
# read data to detect encoding
with open(METADATA_PATH, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))

print(f"Detected metadata encoding: {result['encoding']}")

# read with detected encoding
metadata = pd.read_csv(METADATA_PATH, encoding=result['encoding'], sep=';', index_col='No.')

Detected metadata encoding: Windows-1252


# 💾 **Load Datasets**

Here we load the main train, test, metadata, weather, and solar irradiance datasets. Our goal is not only to import the files but also to validate their structure, identify missing values, inspect column types, and ensure that each dataset aligns with the others.

At the end of this section, we export the processed intermediate outputs to the `data/processed` directory so that the cleaned data can be reused without re-running the full ingestion pipeline.

## **Solar Irradiance Datasets**

Solar irradiance is one of the most influential environmental drivers for energy generation. It measures incoming solar power per unit area and directly affects the system's ability to produce baseline energy. Variations in irradiance depend on atmospheric conditions, cloud coverage, time of day, and seasonal cycles. Incorporating irradiance signals allows the predictive model to capture high-resolution energy-related fluctuations that are not visible in the timestamp alone.

The dataset is split into multiple yearly files, each containing detailed measurements recorded over one calendar year. These files must be combined into a cohesive structure before integrating them into the final feature table.


### **Metadata**

The metadata provides descriptive information for the irradiance dataset, such as field definitions, measurement descriptions, and contextual attributes. Understanding these components ensures accurate interpretation of the irradiance values and proper alignment with the main train and test timestamps.

In [246]:
solar_metadata = metadata[metadata['Variable Name'].isin(solar_irradiance.columns)].reset_index(drop=True)
solar_metadata

,Variable Name,Description
0,Timestamp,Data collection time on solar energy plants.
1,DHI,Diffuse Horizontal Irradiance (DHI) is the quantity of solar radiation received per unit area by a horizontal surface from all directions except the sun which has been scattered by molecules and particles in the atmosphere. (Watt per square metre)
2,DNI,Direct Normal Irradiance (DNI) is the quantity of solar radiation received per unit area by a surface maintained perpendicular to the rays that come in a straight line from the Sun's current position in the sky. (Watt per square metre)
3,GHI,Global Horizontal Irradiance (GHI) is the total amount of solar radiation received per unit area by a horizontal surface. This value includes direct and diffused sunlight. (Watt per square metre)
4,Clearsky DHI,DHI radiation under clear sky conditions. (Watt per square metre)
5,Clearsky DNI,DNI radiation under clear sky conditions. (Watt per square metre)
6,Clearsky GHI,GHI radiation under clear sky conditions. (Watt per square metre)
7,Cloud Type,"Classification of cloud types at the time of measurement: Clear, Probably Clear, Fog, Water, Super-Cooled Water, Mixed, Opaque Ice, Cirrus, Overlapping, Overshooting, Unknown, Dust, Smoke"
8,Dew Point,The temperature at which the air becomes saturated with moisture and water vapor begins to condense. (Celcius)
9,Solar Zenith Angle,The angle between the sun’s rays and the vertical direction other than the solar elevation angle. (Degree)


### **Loading**

Because the irradiance data is split across four separate yearly CSV files (2014–2017), we implement a helper function to load all years automatically. This function standardizes the column formats, concatenates all years into a single dataframe, and ensures that datetime components are compatible across files.

In [247]:
# To load multiple csv file by year
def load_yearly_csv(path, prefix, years) -> pd.DataFrame:
    return pd.concat(
        [pd.read_csv(path / f"{prefix}_{year}.csv") for year in years],
        ignore_index=True
    )

solar_irradiance = load_yearly_csv(SOLAR_IRRADIANCE, 'Solar_Irradiance', range(2014,2018))
solar_irradiance.head()

,Year,Month,Day,Hour,Minute,DHI,DNI,GHI,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Solar Zenith Angle,Surface Albedo,Wind Speed,Relative Humidity,Temperature,Pressure
0,2014,1,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,Probably Clear,-6,124.02,0.12,3.5,78.43,-3,1010
1,2014,1,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,Probably Clear,-5,135.09,0.12,3.8,81.74,-3,1010
2,2014,1,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,Probably Clear,-5,145.77,0.12,4.2,81.25,-3,1010
3,2014,1,1,3,0,0.0,0.0,0.0,0.0,0.0,0.0,Probably Clear,-6,155.07,0.12,4.5,74.14,-3,1010
4,2014,1,1,4,0,0.0,0.0,0.0,0.0,0.0,0.0,Probably Clear,-8,160.55,0.12,4.6,72.80,-4,1010


During inspection, we observe missing values across several irradiance features. These gaps may arise from sensor calibration issues, temporary equipment downtime, extreme weather conditions, or measurement dropouts during periods of low sunlight.

Identifying missingness early allows us to determine appropriate handling strategies such as interpolation, forward/backward filling, climatology-based substitution, or explicit encoding using missing-value indicators. The extent of missing values is summarized in this section before proceeding to timestamp reconstruction.

In [248]:
solar_missing = column_with_missing(solar_irradiance)

print('Solar dataset missing value counts: ')
solar_irradiance[solar_missing].isna().sum()

Solar dataset missing value counts: 


DHI             2777
DNI             2777
GHI             2777
Clearsky DHI    2777
Clearsky DNI    2777
Clearsky GHI    2777
Cloud Type      2486
dtype: int64

The raw irradiance dataset stores date and time in separate columns. For machine learning workflows, especially those dependent on temporal ordering, it is necessary to combine these into a single standardized timestamp. We merge the components into a unified datetime field that can be indexed, filtered, and merged with the train, test, and weather datasets. This timestamp becomes the anchor point for integrating all feature sources into a consolidated modeling table.

In [249]:
# Make timestamp from 5 columns in solar_irradiance: ['Year', 'Month', 'Day', 'Hour', 'Minute']
timestamp = pd.to_datetime(
    solar_irradiance[['Year', 'Month', 'Day', 'Hour', 'Minute']]
)

solar_irradiance.insert(loc=0, column='Timestamp', value=timestamp)

# Drop unnecessary columns
solar_irradiance.drop(columns = ['Year', 'Month', 'Day', 'Hour', 'Minute'], inplace=True)

print('Solar dataset shape: ', solar_irradiance.shape)
solar_irradiance.head()

Solar dataset shape:  (35064, 15)


,Timestamp,DHI,DNI,GHI,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Solar Zenith Angle,Surface Albedo,Wind Speed,Relative Humidity,Temperature,Pressure
0,2014-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,Probably Clear,-6,124.02,0.12,3.5,78.43,-3,1010
1,2014-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,Probably Clear,-5,135.09,0.12,3.8,81.74,-3,1010
2,2014-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,Probably Clear,-5,145.77,0.12,4.2,81.25,-3,1010
3,2014-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,Probably Clear,-6,155.07,0.12,4.5,74.14,-3,1010
4,2014-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,Probably Clear,-8,160.55,0.12,4.6,72.80,-4,1010


## **Weather Dataset**

The weather dataset adds crucial environmental context such as temperature, humidity, wind parameters, and astronomical features like sunrise, sunset, moonrise, and moonset. These attributes help the model understand atmospheric and celestial conditions that may influence baseline energy generation patterns.

Some entries contain mixed or irregular formats such as `"No moonrise"` or `"No moonset"` which we normalize and convert into valid timestamps where applicable. This section documents the inspection, cleaning, missing-value handling, and timestamp alignment for the weather dataset before merging it with the main dataframe.

### **Metadata**

The weather metadata provides structural and descriptive information about the atmospheric variables included in the weather dataset. These features capture daily environmental conditions such as temperature, humidity, dew point, cloud coverage, and astronomical events related to sunrise, sunset, moonrise, and moonset. Understanding the purpose and definition of each feature is essential for integrating them into the main modeling framework, as they represent external physical factors that can influence energy generation patterns. The metadata acts as a reference for interpreting these columns and ensuring that their units and meanings remain consistent throughout preprocessing.

In [250]:
weather_metadata = metadata[metadata['Variable Name'].isin(weather.columns)].reset_index(drop=True)
weather_metadata

,Variable Name,Description
0,Timestamp,Data collection time on solar energy plants.
1,maxtempC,Maximum temperature point of a region during a specific period. (Celsius)
2,mintempC,Minimum temperature point of a region during a specific period. (Celsius)
3,totalSnow_cm,Total accumulated snow height. (Centimetres)
4,sunHour,Estimated total hours of sunlight.
5,uvIndex,UV light index generated by the sun.
6,moon_illumination,Moon illumination percentage (percentage of the moon's surface exposed to sunlight).
7,moonrise,Moonrise time.
8,moonset,Moonset time.
9,sunrise,Sunrise time.


### **Loading**

The weather dataset is provided with a `date_time` column that represents the timestamp of each observation. During loading, we convert this column into a standardized `datetime` format to ensure compatibility with the train, test, and irradiance datasets. After conversion, we rename the column to **Timestamp** so that all datasets use a consistent naming convention, allowing seamless merging based on temporal alignment.

In [251]:
# Load and make timestamp column for weather datasets
weather = pd.read_csv(WEATHER_PATH)
weather['date_time'] = pd.to_datetime(weather['date_time'])

weather.rename(columns={'date_time': 'Timestamp'}, inplace=True)
weather.head()

,Timestamp,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
0,2014-01-01 00:00:00,-3,-6,0.0,8.7,2,1,6:58 AM,5:05 PM,7:12 AM,...,-11,28,3,58,0.0,1022,-4,10,273,20
1,2014-01-01 01:00:00,-3,-6,0.0,8.7,2,1,6:58 AM,5:05 PM,7:12 AM,...,-12,31,4,58,0.0,1023,-5,10,274,22
2,2014-01-01 02:00:00,-3,-6,0.0,8.7,2,1,6:58 AM,5:05 PM,7:12 AM,...,-12,34,5,58,0.0,1024,-5,10,276,25
3,2014-01-01 03:00:00,-3,-6,0.0,8.7,2,1,6:58 AM,5:05 PM,7:12 AM,...,-13,36,6,58,0.0,1024,-5,10,278,27
4,2014-01-01 04:00:00,-3,-6,0.0,8.7,2,1,6:58 AM,5:05 PM,7:12 AM,...,-13,32,9,57,0.0,1025,-6,10,271,23


In [252]:
weather_missing = column_with_missing(weather)

print('Weather dataset missing value counts: ')
weather[weather_missing].isna().sum()

Weather dataset missing value counts: 


Series([], dtype: float64)

We then inspect the astronomical columns—including sunrise, sunset, moonrise, and moonset—to check for entries that do not contain valid times. Some rows include values such as `"No sunrise"`, `"No sunset"`, `"No moonrise"`, or `"No moonset"`. These cases are expected in real-world astronomical data: depending on the season and geographic location, the sun or moon sometimes does not rise or set on certain days. As such, these values are not errors but naturally occurring phenomena that must be treated as missing in the temporal sense.

In [253]:
solar_lunar = ['sunrise', 'sunset', 'moonrise', 'moonset']

for col in solar_lunar:
    print(f'\n{col.capitalize()} columns:')
    print(weather[weather[col].str.contains('No', na=False)][col].value_counts())


Sunrise columns:
Series([], Name: count, dtype: int64)

Sunset columns:
Series([], Name: count, dtype: int64)

Moonrise columns:
moonrise
No moonrise    1176
Name: count, dtype: int64

Moonset columns:
moonset
No moonset    1200
Name: count, dtype: int64


Since these columns originally contain only time-of-day strings without year, month, or day information, we augment them by extracting the year, month, and day from the main Timestamp column. This enables us to reconstruct complete datetime fields for sunrise, sunset, moonrise, and moonset by combining date components with their respective time strings. Doing so simplifies temporal analysis and allows for direct timestamp-based merging with other datasets.

In [254]:
# To add year, month, and day to solar and lunar features
def merge_column_with_timestamp(df, date_column, time_column):
    date_str = df[date_column].dt.strftime('%Y-%m-%d')
    combined = date_str + ' ' + df[time_column].astype(str)

    # No format=, biarkan pandas deteksi sendiri
    return pd.to_datetime(combined, errors='coerce')

for feature in solar_lunar:
    weather[feature] = merge_column_with_timestamp(weather, 'Timestamp', feature)

print('Weather dataset shape: ', weather.shape)
weather.head()

Weather dataset shape:  (35064, 24)


,Timestamp,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
0,2014-01-01 00:00:00,-3,-6,0.0,8.7,2,1,2014-01-01 06:58:00,2014-01-01 17:05:00,2014-01-01 07:12:00,...,-11,28,3,58,0.0,1022,-4,10,273,20
1,2014-01-01 01:00:00,-3,-6,0.0,8.7,2,1,2014-01-01 06:58:00,2014-01-01 17:05:00,2014-01-01 07:12:00,...,-12,31,4,58,0.0,1023,-5,10,274,22
2,2014-01-01 02:00:00,-3,-6,0.0,8.7,2,1,2014-01-01 06:58:00,2014-01-01 17:05:00,2014-01-01 07:12:00,...,-12,34,5,58,0.0,1024,-5,10,276,25
3,2014-01-01 03:00:00,-3,-6,0.0,8.7,2,1,2014-01-01 06:58:00,2014-01-01 17:05:00,2014-01-01 07:12:00,...,-13,36,6,58,0.0,1024,-5,10,278,27
4,2014-01-01 04:00:00,-3,-6,0.0,8.7,2,1,2014-01-01 06:58:00,2014-01-01 17:05:00,2014-01-01 07:12:00,...,-13,32,9,57,0.0,1025,-6,10,271,23


In [255]:
weather_missing = column_with_missing(weather)

print('Weather dataset missing value counts: ')
weather[weather_missing].isna().sum()

Weather dataset missing value counts: 


moonrise    1176
moonset     1200
dtype: int64

After reconstructing the datetime-like features, we re-evaluate missing values. The number of missing entries aligns exactly with the rows previously identified as having `"No sunrise"`, `"No sunset"`, `"No moonrise"`, or `"No moonset"`. This confirms that the missingness originates from natural astronomical conditions rather than data corruption. These missing values are therefore valid and expected, and can be handled appropriately during modeling depending on whether the presence or absence of these events carries predictive significance.

## **Train and Test Dataset**

The train and test datasets serve as the central components of this project, containing the primary target variable and the timestamps that anchor all auxiliary features. Initially, each dataset consists of only two columns:  `Timestamp` and `% Baseline`

### **Metadata**

In [256]:
train_metadata = metadata[metadata['Variable Name'].isin(train.columns)].reset_index(drop=True)
train_metadata

,Variable Name,Description
0,Timestamp,Data collection time on solar energy plants.
1,% Baseline,Percentage of energy generated in one hour based on the energy storage capacity.


### **Loading**

In [257]:
# Load the train and test dataset
test  = pd.read_csv(TEST_PATH)
train = pd.read_csv(TRAIN_PATH)

print('Train Head: ')
print(train.head())

print('\nTest Head: ')
print(test.head())

Train Head: 
          Timestamp  % Baseline
0   Jan 1, 2014 7am      0.0079
1   Jan 1, 2014 8am      0.1019
2   Jan 1, 2014 9am      0.3932
3  Jan 1, 2014 10am      0.5447
4  Jan 1, 2014 11am      0.5485

Test Head: 
          Timestamp  % Baseline
0   Oct 1, 2017 6am         NaN
1   Oct 1, 2017 7am         NaN
2   Oct 1, 2017 8am         NaN
3   Oct 1, 2017 9am         NaN
4  Oct 1, 2017 10am         NaN


Since these files contain no additional predictors, our preprocessing workflow focuses on enriching them by merging external feature sources: solar irradiance, weather attributes, and astronomical information based on their shared temporal index. After loading the train and test files, we perform a full timestamp-based merge with the processed irradiance and weather datasets, forming consolidated feature tables for both training and inference.

In [258]:
# Define feature datasets
features = (weather, solar_irradiance)

# To merge main dataset to multiple feature datasets
def merge_with_features(df, *feature_dfs) -> pd.DataFrame:
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], format="mixed")

    for feature in feature_dfs:
        df = df.merge(feature, on="Timestamp", how="left")

    return df

# Merge main dataset with features
train_merged = merge_with_features(train, *features)
test_merged = merge_with_features(test, *features)

# Print train and test shape
print('Train shape:', train_merged.shape)
print('Test shape:', test_merged.shape)

Train shape: (18942, 39)
Test shape: (1077, 39)


Following the merge, we examine missing values across all integrated features. Compared to the raw solar and weather datasets, the amount of missingness in the merged train and test tables is considerably smaller. This occurs because the train and test timestamps span only a subset of the temporal range covered in the supporting datasets, meaning not all irradiance or weather records are relevant or required.

In [259]:
missing_df = pd.DataFrame({
    'train_missing': train_merged.isna().sum(),
    'test_missing':  test_merged.isna().sum()
})

missing_df = missing_df[(missing_df['train_missing'] > 0) | (missing_df['test_missing'] > 0)]

print('Train test missing value counts:\n\n', missing_df, sep='')

Train test missing value counts:

              train_missing  test_missing
% Baseline                0          1077
moonrise                629            34
moonset                 639            33
DHI                    1044           167
DNI                    1044           167
GHI                    1044           167
Clearsky DHI           1044           167
Clearsky DNI           1044           167
Clearsky GHI           1044           167
Cloud Type              977           112


These missing entries originate from two main sources. First, irradiance sensors occasionally produce incomplete readings due to environmental conditions or measurement interruptions. Second, astronomical variables such as moonrise and moonset are naturally absent on certain days depending on celestial positioning, making missing values expected and physically valid. At this stage, we retain these missing entries for further analysis, where they can be imputed, encoded, or leveraged directly depending on their predictive relevance.

# 📥 **Export Merged Datasets**

After completing the full preprocessing workflow and merging all relevant features into the train and test tables, we export the resulting `train_merged` and `test_merged` dataframes for future use. This ensures that downstream experiments such as model development, feature engineering, and validation—can be performed efficiently without repeating the entire loading and cleaning pipeline. Both datasets are saved in the `data/processed/` directory

In [262]:
train_merged.to_csv(TRAIN_PATH_PROCESSED, index=False)
test_merged.to_csv(TEST_PATH_PROCESSED, index=False)

# 🪨 **Next Steps: EDA and Feature Engineering**

With the merged datasets prepared, the next step is to conduct exploratory data analysis (EDA) to uncover hidden patterns, validate assumptions, and understand how environmental and irradiance features relate to % Baseline. The insights from EDA will guide decisions on cleaning—such as handling missing values, smoothing irregular measurements, or addressing outliers—and inform our feature engineering strategy. Moving forward, we will extract meaningful temporal features, evaluate irradiance and weather-derived signals, and generate additional predictors that enhance the model’s ability to learn energy generation behavior.